<a href="https://colab.research.google.com/github/PranaySaha97/ATPOS-2019-Retinopathy-Eye-Disease-Detection/blob/master/ATPOS_2019_Retinopathy_(DenseNet).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle
api_token = {"username":"Your Userrname","key":"Your Key"}
!mkdir /root/.kaggle
!echo '{"username":"Your Username","key":"Your Key"}' > /root/.kaggle/kaggle.json

!kaggle competitions download -c aptos2019-blindness-detection

import zipfile
zip_ref = zipfile.ZipFile("/content/train_images.zip", 'r')
zip_ref.extractall("/content/train_images/")
zip_ref.close()

zip_ref = zipfile.ZipFile("/content/test_images.zip", 'r')
zip_ref.extractall("/content/test_images")
zip_ref.close()



100% 8.00G/8.01G [01:37<00:00, 101MB/s] 
100% 8.01G/8.01G [01:37<00:00, 88.4MB/s]
 99% 1.48G/1.50G [00:08<00:00, 171MB/s]
100% 1.50G/1.50G [00:08<00:00, 186MB/s]
  0% 0.00/53.7k [00:00<?, ?B/s]
100% 53.7k/53.7k [00:00<00:00, 80.1MB/s]
  0% 0.00/24.5k [00:00<?, ?B/s]
100% 24.5k/24.5k [00:00<00:00, 21.8MB/s]
  0% 0.00/28.3k [00:00<?, ?B/s]
100% 28.3k/28.3k [00:00<00:00, 29.8MB/s]


In [0]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


In [0]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import GlobalAveragePooling2D, Dense, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.applications import DenseNet201

In [0]:
def csv_reader(data): 
    read_data = pd.read_csv(data)
    return(read_data)

In [0]:
train_df = csv_reader('train.csv')
train_df['diagnosis'] = train_df['diagnosis'].astype('str')
train_df['Image_name'] = train_df['id_code'].astype(str)+'.png'
train_df = train_df.drop(columns = ['id_code'])

In [0]:
gen= ImageDataGenerator(rescale=1./255,
                              zoom_range= 0.2,
                              shear_range= 0.2,
                              horizontal_flip= True,
                              vertical_flip = True,
                              validation_split=0.1)
batch_size = 32

In [18]:
training_set=gen.flow_from_dataframe(
    dataframe=train_df,
    directory="train_images",
    x_col="Image_name",
    y_col="diagnosis",
    batch_size=batch_size,
    shuffle=True,
    class_mode="categorical",
    target_size=(224,224),
    subset='training')

testing_set=gen.flow_from_dataframe(
    dataframe=train_df,
    directory="train_images",
    x_col="Image_name",
    y_col="diagnosis",
    batch_size=batch_size,
    shuffle=True,
    class_mode="categorical", 
    target_size=(224,224),
    subset='validation')

Found 3296 validated image filenames belonging to 5 classes.
Found 366 validated image filenames belonging to 5 classes.


In [0]:
def densenet_model():
  dense_mod= DenseNet201(weights='imagenet', include_top=False, input_shape=(224,224,3))
  mod= Sequential()
  mod.add(dense_mod)
  mod.add(GlobalAveragePooling2D())
  mod.add(Dense(2500, activation='relu'))
  mod.add(Dropout(0.5))
  mod.add(Dense(5, activation='sigmoid'))
  return mod

In [17]:
mod= densenet_model()









74842112/74836368 [==============================] - 2s 0us/step


In [19]:
mod.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet201 (Model)          (None, 7, 7, 1920)        18321984  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 1920)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 2500)              4802500   
_________________________________________________________________
dropout_5 (Dropout)          (None, 2500)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 5)                 12505     
Total params: 23,136,989
Trainable params: 22,907,933
Non-trainable params: 229,056
_________________________________________________________________


In [20]:
mod.compile(optimizer=Adam(lr=0.001) , metrics=['accuracy'], loss='categorical_crossentropy')

In [21]:
for data_batch, labels_batch in training_set:
    print('data batch shape:', data_batch.shape)
    print('labels batch shape:', labels_batch.shape)
    break
    
for data_batch, labels_batch in testing_set:
    print('data batch shape:', data_batch.shape)
    print('labels batch shape:', labels_batch.shape)
    break

data batch shape: (32, 224, 224, 3)
labels batch shape: (32, 5)
data batch shape: (32, 224, 224, 3)
labels batch shape: (32, 5)


In [22]:
mod.fit_generator(training_set,
                  epochs = 25,
                  steps_per_epoch = 100,
                  validation_data = testing_set,
                  validation_steps = 10)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/25
100/100 [==============================] - 457s 5s/step - loss: 1.2230 - acc: 0.4544 - val_loss: 7.6774 - val_acc: 0.4875
Epoch 2/25
100/100 [==============================] - 412s 4s/step - loss: 0.7800 - acc: 0.7262 - val_loss: 7.6628 - val_acc: 0.4967
Epoch 3/25
100/100 [==============================] - 404s 4s/step - loss: 0.7215 - acc: 0.7422 - val_loss: 2.2393 - val_acc: 0.4669
Epoch 4/25
100/100 [==============================] - 390s 4s/step - loss: 0.7427 - acc: 0.7313 - val_loss: 7.5211 - val_acc: 0.5132
Epoch 5/25
100/100 [==============================] - 404s 4s/step - loss: 0.6716 - acc: 0.7503 - val_loss: 0.8052 - val_acc: 0.6854
Epoch 6/25
100/100 [==============================] - 397s 4s/step - loss: 0.6187 - acc: 0.7647 - val_loss: 0.7961 - val_acc: 0.7086
Epoch 7/25
100/100 [==============================] - 413s 4s/step - loss: 0.6362 - acc: 0.7556 - val_loss

In [0]:
mod.save('my_mod_dense.h5')

In [0]:
submission = csv_reader('test.csv')
submission['Images'] = submission['id_code'].astype(str)+'.png'

In [28]:
submission.head(5)
submission_datagen=ImageDataGenerator(rescale=1./255)
submission_gen=submission_datagen.flow_from_dataframe(
    dataframe=submission,
    directory="test_images",
    x_col="Images",    
    batch_size=batch_size,
    shuffle=False,
    class_mode=None, 
    target_size=(224,224)
)

Found 1928 validated image filenames.


In [29]:
predictions=mod.predict_generator(submission_gen, steps = len(submission_gen))
print(predictions)

[[1. 0. 0. 0. 1.]
 [1. 0. 0. 0. 1.]
 [1. 0. 0. 0. 1.]
 ...
 [1. 0. 0. 0. 1.]
 [1. 0. 0. 0. 1.]
 [1. 0. 0. 0. 1.]]


In [30]:
max_probability = np.argmax(predictions,axis=1) 
print(max_probability)

[0 0 0 ... 0 0 0]


In [0]:
submission.drop(columns=['Images'], inplace= True)
submission['diagnosis'] = max_probability
submission.to_csv('my_submission.csv', index=False)